In [6]:
!pip install brotli

In [7]:
!pip install schedule

In [8]:
!pip install requests

In [9]:
!pip install openpyxl

In [10]:
import requests
import json
import brotli
import pandas as pd
import schedule
import time

def job():
    # Define the URL and headers
    url = "https://search.vestiairecollective.com/v1/product/search"
    headers = {
        "Accept": "application/json",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "en-US,en;q=0.9",
        "Content-Type": "application/json",
        "Origin": "https://www.vestiairecollective.com",
        "Referer": "https://www.vestiairecollective.com/",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36 Edg/120.0.0.0",
        "X-Usertoken": "anonymous-7cc1f380-028a-4baf-aa91-1e2c3dccca8a"
    }
    total_products = 0
    # Define the payload

    pdata = []
    items_per_page = 48

    # Initialize the offset
    offset = 0

    # Make the POST request for each page
    while True:
        # Update the offset in the payload
        payload = {"pagination":{"offset":offset,"limit":48},
                  "fields":["name","description","brand","model","country","price","discount","link","sold","likes","editorPicks","shouldBeGone","seller","directShipping","local","pictures","colors","size","stock","universeId","createdAt","dutyFree"],
                  "facets":{"fields":["brand","universe","country","stock","color","categoryLvl0","priceRange","price","condition","region","editorPicks","watchMechanism","discount","sold","directShippingEligible","directShippingCountries","localCountries","sellerBadge","isOfficialStore","materialLvl0","size0","size1","size2","size3","size4","size5","size6","size7","size8","size9","size10","size11","size12","size13","size14","size15","size16","size17","size18","size19","size20","size21","size22","size23","categoryLvl1","dutyFree"],
                            "stats":["price"]},
                  "q":None,
                  "sortBy":"relevance","filters":{"categoryLvl0.id":["141"],"universe.id":["2"],
                                                  "catalogLinksWithoutLanguage":["/men-bags/"]},
                  "locale":{"country":"US","currency":"USD","language":"us","sizeType":"US"},
                  "mySizes":None}

        try:
            response = requests.post(url, headers=headers, data=json.dumps(payload))

            # If the response is None or the status code is not 200, break the loop
            if response is None or response.status_code != 200:
                print(f"Request failed with status code {response.status_code if response else 'None'}")
                break

            # Decompress the response content
            decompressed_data = brotli.decompress(response.content)

            # Parse the response
            data = json.loads(decompressed_data)

            # Extract the items
            items = data.get('items', [])

            # If items is empty, we've reached the end, so break the loop
            if not items:
                break
            # Count the number of products
            total_products += len(items)

            # Increment the offset by the number of items per page
            offset += items_per_page
            # Now you can work with the items
            for item in items:
              print(f"{'-'*30}")
              print(f"Name: {item.get('name', 'null')}")
              print(f"Description: {item.get('description', 'null')}")
              print(f"Price: {item.get('price', {}).get('cents', 'null')} {item.get('price', {}).get('currency', 'null')}")
              print(f"Discount Price: {item.get('discount', {}).get('originalPrice', {}).get('cents', 'null')} {item.get('discount', {}).get('originalPrice', {}).get('currency', 'null')}")
              print(f"Likes: {item.get('likes', 'null')}")
              print(f"Picture: https://images.vestiairecollective.com/images/resized/w=480,q=75,f=auto,{item.get('pictures', ['null'])[0]}")  # Assuming there's at least one picture
              print(f"{'-'*30}\n")
              pdata.append({
                    "Name": item.get('name', 'null'),
                    "Description": item.get('description', 'null'),
                    "Price": f"{item.get('price', {}).get('cents', 'null')} {item.get('price', {}).get('currency', 'null')}",
                    "Discount Price": f"{item.get('discount', {}).get('originalPrice', {}).get('cents', 'null')} {item.get('discount', {}).get('originalPrice', {}).get('currency', 'null')}",
                    "Likes": item.get('likes', 'null'),
                    "Picture": f"https://images.vestiairecollective.com/images/resized/w=480,q=75,f=auto,{item.get('pictures', ['null'])[0]}"  # Assuming there's at least one picture
                })


            # Increment the offset by the number of items per page
            print(offset)

        except Exception as e:
            print(f"the end")
            break

    print(f"Total number of products: {total_products}")
    # Convert the list to a DataFrame
    df = pd.DataFrame(pdata)
    # Create a DataFrame for the final offset and total number of products
    df_final = pd.DataFrame([{"Name": f"Final Offset: {offset}", "Description": f"Total Products: {total_products}"}])

    # Append df_final to df
    df = pd.concat([df, df_final], ignore_index=True)

    # Save the DataFrame to an Excel file
    df.to_excel("bagsformen.xlsx", index=False)
while True:
    job()
    time.sleep(10)
    print(f"{'-'*30}","starting over")

the end
Total number of products: 0
------------------------------ starting over
the end
Total number of products: 0
------------------------------ starting over
the end
Total number of products: 0
------------------------------ starting over
the end
Total number of products: 0
------------------------------ starting over
the end
Total number of products: 0
------------------------------ starting over
the end
Total number of products: 0


KeyboardInterrupt: 